<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/snntorch_alpha_w.png?raw=true' width="400">

# snnTorch - Spike Generation and Visualization with `spikegen` and `spikeplot`
## Tutorial 1
### By Jason K. Eshraghian (www.jasoneshraghian.com)

<a href="https://colab.research.google.com/github/jeshraghian/snntorch/blob/tutorials/examples/tutorial_1_spikegen.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction
In this tutorial, you will learn how to use snnTorch to:
* convert datasets into spiking datasets using various encoding methods, 
* how to visualise them, 
* and how to generate random spike trains.

If running in Google Colab:
* You may connect to GPU by checking `Runtime` > `Change runtime type` > `Hardware accelerator: GPU`
* Next, install the latest PyPi distribution of snnTorch by clicking into the following cell and pressing `Shift+Enter`.

In [ ]:
!pip install snntorch

## 1. Setting up the MNIST Dataset

### 1.1. Import packages and setup environment

In [ ]:
import snntorch as snn
import torch

Let's define a few variables.

`data_path` will be used as the target directory for downloading the training set.

`valid_split` will be used to assign data from the training set to the validation set.
*E.g., for a split of 0.1, the validation set will be made up of 10% of the train set.*

`subset` is used to partition the training and test sets down by the given factor.
*E.g., for a subset of 10, a training set of 60,000 will be reduced to 6,000.*

`num_steps` is the number of time steps to simulate.

In [ ]:
# Training Parameters
batch_size=128
data_path='/data/mnist'
val_split = 0.1
subset = 10
num_classes = 10

# Temporal Dynamics
num_steps = 100

# Torch Variables
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### 1.2 Download Dataset

MNIST does not have a specified validation set by default. So we can make a copy of the training set in `mnist_val`.
We won't use `mnist_val` or `mnist_test` here - they're only to demonstrate creating a train-validation split.

In [ ]:
from torchvision import datasets, transforms

# Define a transform
transform = transforms.Compose([
            transforms.Resize((28,28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_val = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

If the above code blocks throws an error, e.g. the MNIST servers are down, then uncomment the following code instead.

In [ ]:
# # temporary dataloader if MNIST service is unavailable
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

# mnist_train = datasets.MNIST(root = './', train=True, download=True, transform=transform)
# mnist_val = datasets.MNIST(root = './', train=True, download=True, transform=transform)
# mnist_test = datasets.MNIST(root = './', train=False, download=True, transform=transform)

`snntorch.utils` contains a few useful functions for modifying datasets.
A train-validation split can be created by calling `valid_split`:

In [ ]:
from snntorch import utils

mnist_train, mnist_val = utils.valid_split(mnist_train, mnist_val, val_split)

Until we actually start doing some training, we won't need large datasets.
So let's make our life simpler by reducing the size of the MNIST dataset.
We can apply `data_subset` to reduce the dataset by the factor given in the argument `subset`.


In [ ]:
mnist_train = utils.data_subset(mnist_train, subset)
mnist_val = utils.data_subset(mnist_val, subset)
mnist_test = utils.data_subset(mnist_test, subset)

To verify, we can take a look at the length of each of our datasets:

In [ ]:
print(f"The size of mnist_train is {len(mnist_train)}")
print(f"The size of mnist_val is {len(mnist_val)}")
print(f"The size of mnist_test is {len(mnist_test)}")

### 1.3 Create DataLoaders
The Dataset objects we created above load training/validation/test data into memory, and the DataLoader will fetch data from this dataset and serve it up in batches. 

DataLoaders in PyTorch are a handy interface for passing data into a network. They return an iterator divided up into mini-batches of size ``batch_size``.

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True)

## 2. Spike Encoding
Spiking Neural Networks (SNNs) are made to exploit time-varying data. And yet, MNIST is not a time-varying dataset. 
This means that we have one of two options for passing input data into an SNN:

1. For a single training sample $x^{(i)}$, directly feed the same static input features at each time step, where each element of $x^{(i)}$ takes on an analog value $x^{(i)}_n \in$  [0, 1]. $n$ spans the number of inputs. 
 This is like converting MNIST into a static, unchanging video.

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_1_static.png?raw=true' width="800">
</center>

2. Convert the input into a spike train of sequence length `num_steps`, where $x^{(i)}$ takes on a discrete value $x^{(i)}
 \in$ {0, 1}.
In this case, MNIST would become a time-varying sequence of spikes that are related to the original image.

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_2_spikeinput.png?raw=true' width="800">
</center>

The first method is quite straightforward, so let's consider (2) in more detail.

The module `snntorch.spikegen` contains a series of functions that simplify the conversion of data into spikes. There are currently three options available for spike generation in `snntorch`:

1. Rate coding: [`spikegen.rate`](https://snntorch.readthedocs.io/en/latest/snntorch.spikegen.html#snntorch.spikegen.rate)
2. Latency coding: [`spikegen.latency`](https://snntorch.readthedocs.io/en/latest/snntorch.spikegen.html#snntorch.spikegen.latency)
3. Delta modulation: [`spikegen.delta`](https://snntorch.readthedocs.io/en/latest/snntorch.spikegen.html#snntorch.spikegen.delta)

*Rate coding* uses input features to determine spiking **frequency**. *Latency coding* uses input features to determine spike **timing**. *Delta modulation* uses the temporal **change** of input features to generate spikes.


### 2.1 Rate coding of MNIST

Each input feature is used as the probability an event occurs, sampled from a binomial distribution. Formally, **X** is a matrix of random variables and **X**~$B(n=1, p=x^{(i)})$ where the
**expected value** $\mathbb E[$**X**$]=x^{(i)}$ is simply the probability that a spike is generated at any given time step.

For an MNIST image, this probability corresponds to the pixel value. A white pixel corresponds to a 100% probability of spiking, and a black pixel will never generate a spike.


<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_3_spikeconv.png?raw=true' width="1000">
</center>

In [ ]:
from snntorch import spikegen

# Iterate through minibatches
data = iter(train_loader)
data_it, targets_it = next(data)
data_it = data_it.to(device)
targets_it = targets_it.to(device)

# Spiking Data
spike_data = spikegen.rate(data_it, num_steps=num_steps, gain=1, offset=0)

As you can see, `spikegen.rate` takes a few arguments that can modify spiking probability:

* `gain` multiplies the input by the given factor, and
* `offset` applies a level-shift to the input.

If the result falls outside of [0,1], this no longer represents a probability. The result will automatically be clipped such that the feature represents a probability.

There are numerous other options available for data conversion available. Fore more detail on converting input features (and targets) to spikes, please [refer to the documentation of `snntorch.spikegen` here](https://snntorch.readthedocs.io/en/latest/snntorch.spikegen.html#snntorch.spikegen.targets_to_spikes).

The structure of the input data is ``[num_steps x batch_size x input dimensions]``:

In [ ]:
print(spike_data.size())

### 2.2 Visualization

#### 2.2.1 Animation
snnTorch contains a module [`snntorch.spikeplot`](https://snntorch.readthedocs.io/en/latest/snntorch.spikeplot.html) that can simplify the process of visualizing, plotting, and animating spiking neurons.


In [ ]:
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML

To plot one sample of data, we have to index into the batch (B) dimension of `spike_data`, ``[T x B x 1 x 28 x 28]``:

In [ ]:
spike_data_sample = spike_data[:, 0, 0]
print(spike_data_sample.size())

`spikeplot.animator` makes it super simple to animate 2-D data:

In [ ]:
fig, ax = plt.subplots()
anim = splt.animator(spike_data_sample, fig, ax)

HTML(anim.to_html5_video())

In [ ]:
# If you're feeling sentimental, you can save the animation: .gif, .mp4 etc.
anim.save("spike_mnist_test.mp4")

The associated target label can be indexed as follows:

In [ ]:
print(f"The corresponding target is: {targets_it[0]}")

As a matter of interest, let's do that again but with 25% of the gain to promote sparsity. This time, we won't bother passing the targets into `spikegen.rate`, as we don't need it.

In [ ]:
spike_data = spikegen.rate(data_it, num_steps=num_steps, gain=0.25)

spike_data_sample2 = spike_data[:, 0, 0]
fig, ax = plt.subplots()
anim = splt.animator(spike_data_sample2, fig, ax)
HTML(anim.to_html5_video())

In [ ]:
# Uncomment for optional save
# anim.save("spike_mnist_test2.mp4")

Now let's average the spikes out over time and reconstruct the input images.

In [ ]:
plt.figure(facecolor="w")
plt.subplot(1,2,1)
plt.imshow(spike_data_sample.mean(axis=0).reshape((28,-1)).cpu(), cmap='binary')
plt.axis('off')
plt.title('Gain = 1')

plt.subplot(1,2,2)
plt.imshow(spike_data_sample2.mean(axis=0).reshape((28,-1)).cpu(), cmap='binary')
plt.axis('off')
plt.title('Gain = 0.25')

plt.show()

The case where `gain=0.25` is lighter than where `gain=1`, as spiking probability has been reduced by a factor of x4.

#### 2.2.2 Raster Plots
Alternatively, we can generate a raster plot of an input sample. This requires reshaping our sample into a 2-D tensor, where the number of steps is the first dimension. We then pass this sample into the function `spikeplot.raster`. 

In [ ]:
# Reshape
spike_data_sample2 = spike_data_sample2.reshape((num_steps, -1))

# raster plot
fig = plt.figure(facecolor="w", figsize=(10, 5))
ax = fig.add_subplot(111)
splt.raster(spike_data_sample2, ax, s=1.5, c="black")

plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

We can also index into one single neuron. Below, we are indexing into the 210th neuron.
Depending on your input data, you may need to index into a few different neurons between 0 & 784 before finding one that spikes.


In [ ]:
fig = plt.figure(facecolor="w", figsize=(8, 1))
ax = fig.add_subplot(111)

splt.raster(spike_data_sample.reshape(num_steps, -1)[:,210].unsqueeze(1), ax, s=100, c="black", marker="|")

plt.title("Input Neuron")
plt.xlabel("Time step")
plt.yticks([])
plt.show()

The idea of rate coding is actually quite controversial. Multiple spikes are needed to achieve any sort of task, and each spike consumes power. It is unlikely to be the only mechanism within the brain, which is both resource-constrained and highly efficient.

We know that the reaction time of a human is around 250ms. If the averaging firing rate of a neuron in the human brain is on the order of 10Hz, then we can only process about 2 spikes within our reaction timescale.

On the other hand, biological neurons are somewhat stochastic. In fact,  neurons fail to fire around 70% of the time that our idealized models would have us believe. Spike rate coding offsets the power disadvantage by showing huge noise robustness: it's fine if some of the spikes fail to generte, because there will be plenty more where they came from.

Rate coding is almost certainly working in conjunction with other encoding schemes in the brain. We'll consider these other encoding mechanisms in the following sections. 

This covers the `spikegen.rate` function. Further information on [can be found in the documentation here](https://snntorch.readthedocs.io/en/latest/snntorch.spikegen.html).



### 2.3 Latency Coding of MNIST
Temporal codes capture information about the precise firing time of neurons; a single spike carries much more meaning than in rate codes which rely on firing frequency.

While this opens up more susceptibility to noise, it can also decrease the power consumed by the hardware running SNN algorithms by orders of magnitude. 

`spikegen.latency` is a function that allows each input to fire at most **once** during the full time sweep.
Features closer to `1` will fire earlier and features closer to `0` will fire later. I.e., in our MNIST case, bright pixels will fire earlier and dark pixels will fire later. 

By default, spike timing is calculated by setting the input feature as a current injection $I_{in}$ into an RC circuit. This current moves charge onto the capacitor, which increases $V(t)$. We assume that there is a trigger voltage, $V_{thr}$, which once reached, generates a spike. The question then becomes: *for a given input current (and equivalently, input feature), how long does it take for a spike to be generated?*

Starting with Kirchhoff's current law, $I_{in} = I_R + I_C$, the rest of the derivation leads us to a logarithmic relationship between time and the input. 

If you've forgotten circuit theory and/or the following means nothing to you, then don't worry! All that matters is: **big** input means **fast** spike; **small** input means **late** spike.


<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_4_latencyrc.png?raw=true' width="600">
</center>

In [ ]:
spike_data = spikegen.latency(data_it, num_steps=100, tau=5, threshold=0.01)

Some of the arguments include:

* `tau`: by default, the input features are treated as a constant current injected into an RC circuit. `tau` is the RC time constant of the circuit. A higher `tau` will induce slower firing.
* `threshold`: the membrane potential the RC circuit must charge to before it can fire. All features below the threshold are saturated.

#### 2.3.1 Raster plot
We'll start with a raster this time.

In [ ]:
fig = plt.figure(facecolor="w", figsize=(10, 5))
ax = fig.add_subplot(111)
splt.raster(spike_data[:, 0].view(num_steps, -1), ax, s=25, c="black")

plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

# optional save
# fig.savefig('destination_path.png', format='png', dpi=300)

To make sense of your raster plot, you'll notice that high intensity features fire first, whereas low intensity features fire last:

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_5_latencyraster.png?raw=true' width="800">
</center>

The logarithmic code coupled with the lack of diverse input values (i.e., the lack of midtone/grayscale features) causes significant clustering in two areas of the plot.
The bright pixels induce firing at the start of the run, and the dark pixels at the end.
We can increase `tau` to slow down our spike times, or we can linearize the data by setting the optional argument `linear=True`.

In [ ]:
spike_data = spikegen.latency(data_it, num_steps=100, tau=5, threshold=0.01, linear=True)

fig = plt.figure(facecolor="w", figsize=(10, 5))
ax = fig.add_subplot(111)
splt.raster(spike_data[:, 0].view(num_steps, -1), ax, s=25, c="black")
plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

The spread of firing times is much more evenly distributed now. This is achieved by simply linearizing the logarithmic equation according to the rules shown below. Unlike the RC model, there's no physical basis for the model. It's just simpler.

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_6_latencylinear.png?raw=true' width="600">
</center>


But notice all firing occurs within the first ~5 time steps, whereas the simulation range is 100 time steps.
This indicates that we have a lot of redundant time steps doing nothing. This can be solved by either increasing `tau` to slow down the time constant, or setting the optional argument `normalize=True` to span the full range of `num_steps`.

In [ ]:
spike_data = spikegen.latency(data_it, num_steps=100, tau=5, threshold=0.01,
                              normalize=True, linear=True)

fig = plt.figure(facecolor="w", figsize=(10, 5))
ax = fig.add_subplot(111)
splt.raster(spike_data[:, 0].view(num_steps, -1), ax, s=25, c="black")

plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

One major advantage of latency coding over rate coding is the increased sparsity of spikes. If neurons are constrained to firing a maximum of once over the time course of interest, then this promotes low-power operation.

In the scenario shown above, a majority of the spikes occur at the final time step, where the input features fall below the threshold. In a sense, the background of the image holds no useful information to us. 

We can remove these redundant features by setting `clip=True`.

In [ ]:
spike_data = spikegen.latency(data_it, num_steps=100, tau=5, threshold=0.01, 
                              clip=True, normalize=True, linear=True)

fig = plt.figure(facecolor="w", figsize=(10, 5))
ax = fig.add_subplot(111)
splt.raster(spike_data[:, 0].view(num_steps, -1), ax, s=25, c="black")

plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

That looks much better!

#### 2.3.2 Animation
We will run the exact same code block as before to create an animation.

In [ ]:
spike_data_sample = spike_data[:, 0, 0]
print(spike_data_sample.size())

In [ ]:
fig, ax = plt.subplots()
anim = splt.animator(spike_data_sample, fig, ax)

HTML(anim.to_html5_video())

This animation is obviously much tougher to make out in video form, but a keen eye will be able to catch a glimpse of the initial frame where most of the spikes occur.
We can index into the corresponding target value to check what value it is.

In [ ]:
# Save output: .gif, .mp4 etc.
# anim.save("mnist_latency.gif")

In [ ]:
print(targets_it[0])

### 2.4 Delta Modulation
There are theories that the retina is adaptive: it will only process information when there is something new to process. If there is no change in your field of view, then your photoreceptor cells will be much lesss prone to firing. 

That is to say: **biology is event-driven**. Our neurons thrive on change.

As a nifty example, a few researchers have dedicated their lives to designing retina-inspired image sensors, for example, the [Dynamic Vision Sensor](https://ieeexplore.ieee.org/abstract/document/7128412/). Although [the attached link is from over a decade ago, the work in this video](https://www.youtube.com/watch?v=6eOM15U_t1M&ab_channel=TobiDelbruck) was clearly ahead of its time.

Delta modulation is based on event-driven spiking. The `snntorch.delta` function accepts a time-series tensor as input. It takes the difference between each subsequent feature across all time steps. By default, if the difference is both *positive* and *greater than the threshold $V_{thr}$*, a spike is generated:

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial1/1_2_7_delta.png?raw=true' width="600">
</center>

To illustrate, let's first come up with a contrived example where we create our own input tensor.

In [ ]:
# Create a tensor with some fake time-series data
data = torch.Tensor([0, 1, 0, 2, 8, -20, 20, -5, 0, 1, 0])

# Plot the tensor
plt.plot(data)

plt.title("Some fake time-series data")
plt.xlabel("Time step")
plt.ylabel("Voltage (mV)")
plt.show()

Let's pass the above tensor into the `spikegen.delta` function, with an arbitrarily selected `threshold=4`:

In [ ]:
# Convert data
spike_data = spikegen.delta(data, threshold=4)

# Create fig, ax
fig = plt.figure(facecolor="w", figsize=(8, 1))
ax = fig.add_subplot(111)

# Raster plot of delta converted data
splt.raster(spike_data, ax, c="black")

plt.title("Input Neuron")
plt.xlabel("Time step")
plt.yticks([])
plt.xlim(0, len(data))
plt.show()

There are three time steps where the difference between $data[T]$ and $data[T+1]$ is greater than or equal to $V_{thr}=4$. This means there are three on-spikes. 

The large dip to $-20$ has not been captured in our spikes. It might be the case that our data cares about negative swings as well, in which case we can enable the optional argument `off_spike=True`.

In [ ]:
# Convert data
spike_data = spikegen.delta(data, threshold=4, off_spike=True)

# Create fig, ax
fig = plt.figure(facecolor="w", figsize=(8, 1))
ax = fig.add_subplot(111)

# Raster plot of delta converted data
splt.raster(spike_data, ax, c="black")

plt.title("Input Neuron")
plt.xlabel("Time step")
plt.yticks([])
plt.xlim(0, len(data))
plt.show()

We've generated additional spikes, but this isn't actually the full picture! 

If we print out the tensor, we will discover that we have actually generated "off-spikes". These spikes take on a value of `-1`.

In [ ]:
print(spike_data)

Although we have only shown `spikegen.delta` on a fake sample of data, the true intention is to pass in time-series data and only generate an output when there has been a sufficiently large event. 

That wraps up the three main spike conversion functions! There are still additional features to each of the three conversion techniques that have not been detailed in this tutorial. We recommend [referring to the documentation for a deeper dive](https://snntorch.readthedocs.io/en/latest/_modules/snntorch/spikegen.html).

## 3. Spike Generation
Now what if we don't actually have any data to start with? 
Say we just want a randomly generated spike train from scratch.
`spikegen.rate` has a nested function, `rate_conv` which takes care of converting features into spikes.

All we have to do is initialize a randomly generated `torchTensor` to pass in.

In [ ]:
# Create a random spike train
spike_prob = torch.rand((num_steps, 28, 28), device=device, dtype=dtype) * 0.5
spike_rand = spikegen.rate_conv(spike_prob)

### 3.1 Animation

In [ ]:
fig, ax = plt.subplots()
anim = splt.animator(spike_rand, fig, ax)

HTML(anim.to_html5_video())

In [ ]:
# Save output: .gif, .mp4 etc.
# anim.save("random_spikes.gif")

### 3.2 Raster

In [ ]:
fig = plt.figure(facecolor="w", figsize=(10, 5))
ax = fig.add_subplot(111)
splt.raster(spike_rand[:, 0].view(num_steps, -1), ax, s=25, c="black")

plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

# Conclusion

That's it for spike conversion and generation. 
This approach generalizes beyond images, to single-dimensional and multi-dimensional tensors.

For reference, the documentation for [`spikegen` can be found here](https://snntorch.readthedocs.io/en/latest/_modules/snntorch/spikegen.html) and for [`spikeplot`, here](https://snntorch.readthedocs.io/en/latest/_modules/snntorch/spikeplot.html)

In the next tutorial, you will learn the basics of spiking neurons and how to use them. Following that, you will be equipped with the tools to train your own spiking neural network in tutorial 3. 